## Introduction to Jupyter Notebooks and R

Grayson Boyer, January 11, 2021

The kernel should be set to "R" for this tutorial.

Terms:

- **Jupyter Notebook:** a shareable document that may contain blocks of code, equations, text, and visualizations.
- **Jupyter Hub:** An online service for working with Jupyter notebooks. This instance of Jupyter Hub is called the Water Organic Rock Microbe (WORM) Portal.
- **R:** a programming language.
- **CHNOSZ:** a package (add-on) for R that supports thermodynamic calculations.

Execute code in a cell by selecting it and hitting the 'run' button, or SHIFT+ENTER.

In [ ]:
"Hello World!"

You can do math in a cell. '#' allows comments on a line.

In [ ]:
1+1  # addition
2-1  # subtraction
2*3  # multiplication
9/3  # division
3^3  # exponents

In [ ]:
9*10^(4/6)

Concatenate with c() to group things:

In [ ]:
c(6, 2, 3, 3)

In [ ]:
c(6, 2, 3, 3)*2

In [ ]:
seq(2, 7, 1) # get a sequence of numbers (to, from, by)
rep(4, 3)    # repeat a value (value to repeat, times)

## CHNOSZ

### Getting CHNOSZ ready

Get CHNOSZ loaded in your notebook with:

In [ ]:
library(CHNOSZ)

Use info("name") to search for the index number of a chemical species in OBIGT.

In [ ]:
info("Pyri")  # a partial name might result in approximate matches to try

In [ ]:
info("pyrite") # case sensitive!

Use info(index) to get the entry itself.

In [ ]:
info(1899)

What do these columns mean? What units are entries in? We can use `help(thermo)` for documentation.

In [ ]:
#help(thermo)

You can nest two info()s to look up a chemical species directly from its name.

In [ ]:
info(info("pyrite"))

### Perform a SUPCRT-like thermodynamic calculation

Let's calculate reaction properties for the Knallgas reaction:

$$H_{2(aq)} + \frac{1}{2}O_{2(aq)} = H_{2}O_{}$$

We can use CHNOSZ's subcrt() function for this.

In [ ]:
subcrt(
    species=c("H2", "O2", "H2O"), # what chemical species are present in rxn?
    coeff=c(-1, -0.5, 1) # what are their stoichiometries? negative = reactant
      )

What are the units used in the output table? We can use help() to find the answer in the documentation for the subcrt function:

In [ ]:
#help(subcrt)

You can modify subcrt to calculate properties at a specific temperature (T) and pressure (P). If a temperature is given but no pressure, CHNOSZ will perform calculations along the liquid-vapor saturation curve (PSAT).

In [ ]:
subcrt(
    species=c("H2", "O2", "H2O"), # what chemical species are present in rxn?
    coeff=c(-1, -0.5, 1), # what are their stoichiometries? negative = reactant
    T=350, # degrees C
    P=2000 # bars pressure
      )

This can be done for a range of temperatures and pressures, but be certain the number of T's and P's are the same! (See what happens to the results if they're not!)

In [ ]:
subcrt(
    species=c("H2", "O2", "H2O"), # what chemical species are present in rxn?
    coeff=c(-1, -0.5, 1), # what are their stoichiometries? negative = reactant
    T=c(150, 250, 250.25, 350), # degrees C
    P=c(1000, 1050, 1050, 2000) # bars pressure
      )

You can change the phase (cr, liq, aq, gas) of a chemical species if an entry exists in OBIGT. Searching for "O2" with info() will reveal that it has entries for "aq" and "gas" phases. Aqueous O2 is selected by default in subcrt(), but we can change it to a gas in our reaction:

In [ ]:
subcrt(
    species=c("H2", "O2", "H2O"), # what chemical species are present in rxn?
    coeff=c(-1, -0.5, 1), # what are their stoichiometries? negative = reactant
    state=c("aq", "gas", "liq"), # what phases are the chemical species in?
    T=350, # degrees C
    P=2000 # bars pressure
      )

Another example, this time with the reaction:

\begin{align}
\mathop{\rm{K(AlSi_{3})O_{8}}}\limits_{(K\text{-}feldspar)} + \mathop{\rm{Al_{2}Si_{2}O_{5}(OH)_{4}}}\limits_{(kaolinite)} & \rightleftharpoons \rm{H_{2}O} + \rm{2SiO_{2(aq)}} + \mathop{\rm{KAl_{2}(AlSi_{3})O_{10}(OH)_{2}}}\limits_{(muscovite)} \\
\end{align}

First, let's check if these entries are in the database:

In [ ]:
info("K-feld") # a partial name might result in approximate matches to try

In [ ]:
info(info(c("K-feldspar", "kaolinite", "muscovite")))

In [ ]:
# tell CHNOSZ to use species from the SUPCRT92 database
add.OBIGT("SUPCRT92")

In [ ]:
info(info(c("K-feldspar", "kaolinite", "muscovite")))

In [ ]:
subcrt(
    species=c("K-feldspar", "kaolinite", "H2O", "SiO2", "muscovite"), # what chemical species are present in rxn?
    coeff=c(-1, -1, 1, 2, 1), # what are their stoichiometries? negative = reactant
    T=seq(0, 350, 10) # seq(from, to, by) is used to specify a sequence
      )

### Exporting and plotting results from subcrt

You can export a table of results into your notebook directory as a downloadable csv.

In [ ]:
my_results = subcrt(
    species=c("K-feldspar", "kaolinite", "H2O", "SiO2", "muscovite"), # what chemical species are present in rxn?
    coeff=c(-1, -1, 1, 2, 1), # what are their stoichiometries? negative = reactant
    T=seq(0, 350, 10) # seq(from, to, by) is used to specify a sequence
      )

write.csv(my_results$out, "my results.csv")

Results can also be plotted directly in a notebook:

In [ ]:
plot(x=my_results$out$T, # temperature
     y=my_results$out$logK, # logK
     col="red", # line color
     lwd=2, # line thickness
     type="l", # specify line plot
     xlab="Temperature, degrees C", # x label
     ylab="logK" # y label
    )

Export a plot as a png image.

In [ ]:
png("my plot.png", width=4, height=4, units="in", res=300)

plot(x=my_results$out$T, # temperature
     y=my_results$out$logK, # logK
     col="red", # line color
     lwd=2, # line thickness
     type="l", # specify line plot
     xlab="Temperature, degrees C", # x label
     ylab="logK" # y label
    )

dev.off()

## Diagram Demo

In [ ]:
reset() # reset database

In [ ]:
# set plot size options
library(repr)
options(repr.plot.width=4, repr.plot.height=3) # width and height of plots

Activity diagram of aqueous carbonate species at 25 $^{\circ}$C and 1 bar as a function of pH.

$$\text{CO}_{2} + \text{H}_{2}\text{O} \rightleftharpoons \text{HCO}_{3}^{-} + \text{H}^{+} \rightleftharpoons \text{CO}_{3}^{2-} + 2\text{H}^{+}$$

This plot assumes a total carbon log activity of -3

In [ ]:
basis("CHNOS+")
species(c("HCO3-","CO3-2","CO2"))
a <-affinity(pH=c(2,14), T=25)
e <-equilibrate(a)
diagram(e, col=c("red","blue","green"), lwd=2, lty=c(1,1,1), dy=-1)

As above, but with fractional abundances ("degree of formation"):

In [ ]:
basis("CHNOS+")
species(c("HCO3-","CO3-2","CO2"))
a <-affinity(pH=c(2,14), T=25)
e <-equilibrate(a)
diagram(e, alpha=TRUE, ylim=c(0,1), col=c("red","blue","green"), lwd=2, lty=c(1,1,1), dy=-0.1)

Predominance diagram of carbonate species along the liquid-vapor saturation curve as a function of temperature and pH:

In [ ]:
basis("CHNOS+")
species(c("HCO3-","CO3-2","CO2"))
a <-affinity(pH=c(2,14), T=c(0,350))
diagram(a, fill="heat")

Activity diagram of Al$^{3+}$ species at 25 $^{\circ}$C and 1 bar as a function of pH.

In [ ]:
reset() # reset database and basis species

In [ ]:
# load a few additional species like AlO+
add.obigt("SLOP98")

In [ ]:
basis(c("Al+3","H2O","O2","H+"),c(-3,0,-80,-7))
species(c("Al+3","AlOH+2","AlO+","HAlO2", "AlO2-"))
a <-affinity(pH=c(2, 8), T=25)
diagram(a, alpha=TRUE, ylim=c(0,1), col=c("red","blue","green","orange","purple"), lwd=2, lty=c(1,1,1), dy=-0.1)

Predominance diagram of Al$^{3+}$ species as a function of pH and temperature:

In [ ]:
basis(c("Al+3","H2O","O2","H+"),c(-3,0,-80,-7))
species(c("Al+3","AlOH+2","AlO+","HAlO2", "AlO2-"))
a <-affinity(pH=c(0, 8, 600), T=c(25, 350, 600))
diagram(a, fill="heat", main="Al+3 speciation, PSAT pressure") 

a <-affinity(pH=c(0, 8, 600), T=c(25, 350, 600), P=5000)
diagram(a, fill="heat", main="Al+3 speciation, 5000 bars")

Eh-pH diagram of aqueous sulfur species.

In [ ]:
reset() # reset database and basis species

In [ ]:
basis("CHNOS+")
swap.basis("O2", "e-")
species(c("H2S", "HS-", "HSO4-", "SO4-2"))
a <- affinity(pH = c(0, 12), Eh = c(-0.5, 1))
diagram(a, fill = "terrain")
water.lines(a) # adds lines indicating the stability region of water